<h1><center>ASSIGNMENT 3</center></h1>
## TASK 1: Ranking by desirability
Below, we load the data from the new file provided by Zillow. The interesting variables are found out like Assignment 2 and made into a dataframe for further processing.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

df = pd.read_csv("properties_2017.csv")
col_array=list(df)
data_array=(len(df.index)-df.isnull().sum())/len(df.index)
col_selected=[]

for i in range(0,len(data_array)):
    if data_array[i]>=0.5:
        col_selected.append(col_array[i])

remove_labels=['fips','propertyzoningdesc','rawcensustractandblock','censustractandblock','assessmentyear','taxdelinquencyyear']

for x in remove_labels:
    if x in col_selected:
        col_selected.remove(x)
    
selected_df=df[col_selected].copy()

After selecting the interesting columns, we further narrow down our choice. We select a number of columns which are most likely to determine the price of a house and form a scoring function using them. We then score each of the house and find out the 10 houses which score the most and least according to our function.

In [11]:
import math

scoring_cols=['parcelid','bathroomcnt','buildingqualitytypeid','finishedsquarefeet12','unitcnt','yearbuilt','taxamount','latitude','longitude']
temp_df=selected_df[scoring_cols].copy()
temp_df=temp_df.fillna(temp_df.mean())

normalized_df=pd.DataFrame()

for col in scoring_cols:
    if col=='parcelid':
        normalized_df[col]=temp_df[col]
    else:
        normalized_df[col]=(temp_df[col] - temp_df[col].mean()) / (temp_df[col].max() - temp_df[col].min())

def scoring_fn(bath,quality,area,unit,year,tax):
    return (0.05*bath)+(0.3*(1-quality))+(0.3*area)+(0.1*unit)+(0.1*year)+(0.15*tax)
    
normalized_df['score'] = normalized_df.apply(lambda row: scoring_fn(row['bathroomcnt'], row['buildingqualitytypeid'],row['finishedsquarefeet12'],row['unitcnt'],row['yearbuilt'],row['taxamount']), axis=1)

normalized_df=normalized_df.sort_values('score')
print('Best 10 houses:\n')
print(normalized_df.head(10)[['parcelid']].to_string(index=False))
print('\nWorst 10 houses:\n')
print(normalized_df.tail(10)[['parcelid']].to_string(index=False))

Best 10 houses:

parcelid
12545239
11887759
11588667
11641464
11506355
11982343
12527124
11982362
11390237
11661131

Worst 10 houses:

parcelid
 12036477
167638107
 10890406
 13073054
 11857690
 11984395
 11843438
 11746970
167638594
 10944745


### Description of cost function
The columns making up the cost function has been selected according to their potential to play an important factor in determining the pricing of a house. In doing that, the columns had been normalized because each column has a different unit and largely varying range. The weights given to the different columns has been assigned so as to reflect the effect of that factor on the price.
<p>The columns selected for the weight function are the following:</p>
<p>
<strong>Building quality type: </strong>The quality of a building would be very important in determining the price of the building. The higher the quality, the greater would be the price of it. A lower number in the quality column denotes a better building. So the scoring function has been taken to subtract the normalized quality from 1 so that a lower value may reflect highly on the score.
</p>
<p>
<strong>Finished square feet: </strong>Higher the area of the house, the greater will be the price. The importance of the finished square feet in determining the price of a house is reflected in its weight.
</p>
<p>
<strong>Unit count: </strong>A house having more units would have a higher price tag than one with smaller number of units.
</p>
<p>
<strong>Year built: </strong>A house built more recently is more likely to have a higher price than old ones. In assignment 2, we saw that the quality of house is related to the year they were built in.
</p>
<p>
<strong>Tax on property: </strong>Highly taxed houses are also the more expensive ones.
</p>
<p>
<strong>Bathroom count: </strong>Not a very important as seen from the weight. However a house having more bathrooms would also be roomier and so more expensive.
</p>
<p>A large number of factors play into the price of a house. However the factors chosen here are the most likely to affect the price. Their weight has been arranged so that they portray an accurate picture of the relative importance of the different factors involved. As such, the cost function should be very accurate.</p>

## TASK 2: Pairwise distance function

In [21]:
def distance_fn(lat1,lat2,long1,long2,quality1,quality2,area1,area2,unit1,unit2,tax1,tax2):
    lat_diff=math.pow(math.fabs(lat1-lat2),2)
    long_diff=math.pow(math.fabs(long1-long2),2)
    qual_diff=math.pow(math.fabs(quality1-quality2),2)
    area_diff=math.pow(math.fabs(area1-area2),2)
    unit_diff=math.pow(math.fabs(unit1-unit2),2)
    tax_diff=math.pow(math.fabs(tax1-tax2),2)
    return math.sqrt(lat_diff+long_diff+qual_diff+area_diff+unit_diff+tax_diff)    

best_df=normalized_df.tail(1000).copy()
worst_df=normalized_df.head(1000).copy()

count,sum_count=0,0
#print('Pairwise distance between best pairs of houses:\n')
for index1,row1 in best_df.iterrows():
    for index2,row2 in best_df.iterrows():
        if index2==index1:
            continue
        distance_score=distance_fn(row1['latitude'],row2['latitude'],row1['longitude'],row2['longitude'],row1['buildingqualitytypeid'],row2['buildingqualitytypeid'],row1['finishedsquarefeet12'],row2['finishedsquarefeet12'],row1['unitcnt'],row2['unitcnt'],row1['taxamount'],row2['taxamount'])
        sum_count=sum_count+distance_score
        count=count+1.0
        #print(int(row1['parcelid']),'   ',int(row2['parcelid']),'   ',distance_score)
print('\nMean distance between best pairs of houses: ',sum_count/count)

count,sum_count=0,0
#print('\nPairwise distance between best and worst pairs of houses:\n')
for index1,row1 in worst_df.iterrows():
    for index2,row2 in worst_df.iterrows():
        if index2==index1:
            continue
        distance_score=distance_fn(row1['latitude'],row2['latitude'],row1['longitude'],row2['longitude'],row1['buildingqualitytypeid'],row2['buildingqualitytypeid'],row1['finishedsquarefeet12'],row2['finishedsquarefeet12'],row1['unitcnt'],row2['unitcnt'],row1['taxamount'],row2['taxamount'])
        sum_count=sum_count+distance_score
        count=count+1.0
        #print(int(row1['parcelid']),'   ',int(row2['parcelid']),'   ',distance_score)
print('\nMean distance between worst pairs of houses: ',sum_count/count)

count,sum_count=0,0
#print('\nPairwise distance between best and worst pairs of houses:\n')
for index1,row1 in best_df.iterrows():
    for index2,row2 in worst_df.iterrows():
        if index2==index1:
            continue
        distance_score=distance_fn(row1['latitude'],row2['latitude'],row1['longitude'],row2['longitude'],row1['buildingqualitytypeid'],row2['buildingqualitytypeid'],row1['finishedsquarefeet12'],row2['finishedsquarefeet12'],row1['unitcnt'],row2['unitcnt'],row1['taxamount'],row2['taxamount'])
        sum_count=sum_count+distance_score
        count=count+1.0
        #print(int(row1['parcelid']),'   ',int(row2['parcelid']),'   ',distance_score)
print('\nMean distance between best and worst pairs of houses: ',sum_count/count)


Mean distance between best pairs of houses:  0.24920694123627132

Mean distance between worst pairs of houses:  0.12473760923884378

Mean distance between best and worst pairs of houses:  1.0172083307860769


In Task 1, we found out the houses which were most and least desirable. In this task, we come up with a pairwise distance function for different houses and test it on the dataset that we have scored in the previous task. 
<p>The factors which would affect the "distance" of a house from each other are similar to those which affect the desirability of a house. So a number of factors which were considered during calculating the desirability score for each house has been retained while calculating the distance. However, in this case, we also consider the geographic aspect of the house. It is only natural that houses which are geographicaly close by should also be very close to each other in terms of "distance". The following factors have been considered in coming up with a distance function:</p>
<p>
<strong>Latitude: </strong>One of the two factors in the geographic position of a house. Houses which are close by are similar in terms of price and setting. 
</p>
<p>
<strong>Longitude: </strong>The second factor in determining the geographic position of a house.
</p>
<p>
<strong>Building quality: </strong>This is an important factor in the distance function. Houses which have similar quality would most likely be of the same type.
</p>
<p>
<strong>Area: </strong>The area of a house is another factor which determines whether two houses are similar or not. It is usually the case that houses in a certain locality have nearly the same area as they are designed in the same way and the land area is the same in both cases.
</p>
<p>
<strong>Unit count: </strong>Houses having different number of units would most likely be different from each other.
</p>
<p>
<strong>Tax amount: </strong>The tax amount is a good indicator for the price of a house. Some locations have high taxes and the houses in those locations are fairly similar to each other and so should have a smaller distance between them. 
</p>
<p>
The distance function takes into consideration all of these factors and then calculates the Euclidean distance between houses using the normalized values of these factors as coordinates for the houses. We have already ranked the houses in Task 1 and now we can test the distance of houses in and among different groups. We take out the 1000 best houses and the 1000 worst houses from the dataset according to the desirability score and calculate the distance between and among them. The above code prints out three lines: the mean distance among the best houses, the mean distance among the worst houses and the mean distance while a house is taken from the best and another from the worst <em>(to display the pairwise distances in each group, uncomment the three lines in code which print out the pairwise house distances in each group)</em>. As it can be seen, the distance of houses among each group is less compared to when we take the houses from different groups. So the distance function performs fairly well.
</p>

## TASK 3: Clustering